#### ddp的每个gpu都加载一个模型，fsdp的每个gpu加载部分模型参数
`使用gloo通信方式时,在测试过程中可能会出现bug,[../third_party/gloo/gloo/transport/tcp/pair.cc:598] Connection closed by peer [127.0.1.1]:27521`  
`使用nccl通信方式时，在多张4090上训练，在加载完模型后可能会卡住不动，添加环境变量os.environ['NCCL_P2P_DISABLE'] = "1"`

#### 1.初始化环境变量


In [3]:
import os
import torch

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'  # 设置可用的GPU索引
os.environ['WORLD_SIZE'] = str(torch.cuda.device_count()) # 设置进程数，等于前面指定的可用的显卡数

#### 2. 初始化进程组

In [4]:
import torch.distributed as dist

def init_ddp(local_rank):
    dist.init_process_group(backend="nccl", rank=local_rank)

#### 3.销毁进程组


In [ ]:
def cleanup():
    dist.destroy_process_group()

#### 4.运行的主函数

In [2]:

def demo_ddp(rank, world_size): # mp.spawn 自动为每个启动的进程分配一个 rank 参数，rank 是从 0 到 nprocs-1 的整数。
    init_ddp(rank)  # 初始化进程组

    model = SimpleModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    for epoch in range(10):
        optimizer.zero_grad()
        outputs = ddp_model(torch.randn(20, 10).to(rank))
        labels = torch.randn(20, 10).to(rank)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        if rank == 0:
            print(f"Epoch {epoch} Loss: {loss.item()}")

    cleanup()




1

In [ ]:

if __name__ == "__main__":
    mp.spawn(demo_ddp,
             args=(world_size,),
             nprocs=world_size,
             join=True)
    run_demo(demo_ddp, n_gpus)